In [2]:
from caveclient import CAVEclient
from meshparty import trimesh_io, trimesh_vtk
import cloudvolume 
import numpy as np


In [3]:
client = CAVEclient('minnie65_public_v117')


In [4]:
# to access dynamic meshes, you can query the segmentation source from the info client
client.info.segmentation_source()

'graphene://https://minnie.microns-daf.com/segmentation/table/minnie65_public_v117'

In [5]:
# this can be used to initialize a cloudvolume object
cv = cloudvolume.CloudVolume(client.info.segmentation_source(), progress=False)
# which given a root_id can be used to get a mesh
# cloud volume returns a dictionary with meshes as the keys
mesh = cv.mesh.get(864691135474648896)[864691135474648896]


In [6]:
# meshes are triangular meshes with vertices and faces
# vertices are Nx3 x,y,z positions in nm
# faces are Kx3 i,j,k indices into vertices that describe triangles
mesh.vertices.shape, mesh.faces.shape


((1698627, 3), (3388543, 3))

Since downloading meshes can take some time, particularly for these dynamic meshes
it is convient to cache them on disk. Also there a number of analytical techniques that you often want to accomplish on meshes, such as constructing a graph of the vertices and edges and calculating distances on them.  To facilitate this we developed a package called MeshParty. 

In [7]:


# to enable a cache, create a MeshMeta object
mm = trimesh_io.MeshMeta(cv_path = client.info.segmentation_source(),
                         disk_cache_path='minnie65_v117_meshes',
                         map_gs_to_https=True)



now i can get a mesh throught this and it will be cached in memory
and in disk in case i need it again.
restart the kernel and run the below cells again to see the difference.
you'll find the mesh file saved as an hdf5 file in the "minnie65_v117_meshes"
subdirectory

In [8]:
mesh = mm.mesh(seg_id=864691135474648896)

The MeshParty object has more useful properties and attributes
such as a scipy.csgraph sparse graph object (mesh.csgraph) and a networkx 
graph object (mesh.nxgraph) 

Read more about what you can do with MeshParty on its [Documentation](https://meshparty.readthedocs.io/en/latest/?badge=latest).

In particular it lets you associate skeletons and annotations onto the mesh into a "meshwork" object.  

The meshes that are available in the visualizaton on micronsexplorer are fast because they are static and all located in a single file, and because they were downsampled to 3 different levels of downsampling.  This makes them faster to interactive and more useful for many applications.  They also do not have a sense of the morphology of the neuron and so where there are locations of self contact, the mesh is merged. So depending on your application they made be preferrable to you.  Our reccomendations for general application area between the static and dynamic meshes are as follows, and the reasons why

<h2>Dynamic</h2>
* Morphological analysis (avoids self mergers)

* Access historical mesh data during proofreading (not possible in flat)

* Want to download with spatial querying (not presently implemented in cloudvolume)

<h2>Static</h2>
* Visualization (faster, downsampling available)
* Somatic analysis (residual nucleus fragments exist)


In [12]:
# note you need the use_https option here or else cloudvolume will try to use your google credentials
# to access the bucket, and you don't have access to the bucket interface, just anonymous downloading
cv = cloudvolume.CloudVolume("precomputed://gs://iarpa_microns_phase3/minnie65/seg", use_https=True)
# this would also work if you wanted to grab it from the AWS bucket instead
# cv = cloudvolume.CloudVolume("precomputed://s3://bossdb-open-data/microns/minnie/minnie65-flat-seg/", use_https=True)

In [13]:
# the cloud volume interface is the same
mesh = cv.mesh.get(864691135474648896)[864691135474648896]
# but its a faster initial download

In [15]:
# as you can see the meshes aren't exactly the same though
mesh.vertices.shape, mesh.faces.shape

((2017003, 3), (3976580, 3))

In [19]:
# in addition the flat meshes are available in 3 levels of detail
# this covers two orders of magnitude of detail
# this is what neuroglancer leverages to smartly load the data
# at the resolution necessary to render the current scene.  
for lod in range(4):
    mesh = mesh = cv.mesh.get(864691135474648896, lod=lod)[864691135474648896]
    print(f"lod {lod}: n_verts: {mesh.vertices.shape[0]} n_faces: {mesh.faces.shape[0]}")

lod 0: n_verts: 2017003 n_faces: 3976580
lod 1: n_verts: 575851 n_faces: 1121653
lod 2: n_verts: 101194 n_faces: 192476
lod 3: n_verts: 23099 n_faces: 42211
